In [1]:
import os
import re
import ast
import nltk 
import spacy
import pandas as pd
from tqdm import tqdm
from typing import List
from sklearn.preprocessing import LabelEncoder

c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!python -m spacy download fr_core_news_lg

^C


     ------------------------------------ 571.8/571.8 MB 969.6 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')


In [2]:
nlp = spacy.load("fr_core_news_lg")

In [3]:
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

True

In [34]:
current_dir = os.path.dirname(os.path.realpath("__file__"))
repo_dir = os.path.dirname(current_dir)
DATA_FILE = os.path.join(repo_dir,"data","cleaned_data.csv")
data = pd.read_csv(DATA_FILE, encoding="utf-8",sep=',')

In [36]:
print(data["recettes"][0])

soupe veloutee potimarron pommes terre
Ingrédients : 
2 pommes terre 
 2 oignons haches 
  3 gousses ail hachees 
  10 cl creme fraiche 
  1 c.a.c curry 
  1 pincee muscade 
 1 feuille laurier 
 1 potimarron 
 1 cube bouillon poule
Ustensiles :
louche mixeur plongeant blender chauffant couteau econome poele couvercle mixeur mijoteuse electrique balance cuisine
Instructions :
 enlever ecorce pepins potimarron obligatoire peler cas choisir bio bien laver couper chair gros morceaux 
 eplucher pommes terre couper gros morceaux 
 faire revenir oignons ail haches beurre feu doux 
 ajouter pommes terre potimarron faire revenir 5 min 
 couvrir eau ajouter bouillon poule emiette curry muscade feuille laurier mijoter 30 min feu doux 
 legumes cuits verifier aide couteau mixer preparation 
 verifier assaisonnement saler poivrer gout ajouter creme fraiche mijoter 2 3 min bon appetit


In [5]:
data.sample(5)

,Nom,Duree,Personnes,Type,Ingredients,Ustensiles,Etapes
24069,"Pâtes à la ""carbonara"" à la française",20 min,4,Plat principal,"['500g', 'de pâtes ', '50cl', 'de crème fraîch...","['Top 5 des meilleures passoires', 'Set 3 poêl...","[{'Étape 1': ""Cuire les pâtes dans un grand vo..."
10684,Tarte aux poireaux et aux lardons,50 min,6,Entrée,"['250g', 'de farine ', '140g', 'de beurre ', '...","['Moule à tarte', 'Four', 'couteau', 'pinceau'...",[{'Étape 1': 'Préchauffer le four à 210°C (the...
31615,Flan pâtissier traditionnel,1 h,8,Dessert,"['11', 'pâte brisée ', '1l', 'de lait ', '150g...","['Tamis', 'Cuillère en bois', 'Four', 'Moule à...",[{'Étape 1': 'Faire bouillir le lait avec la g...
3712,Salade César,20 min,4,Entrée,"['25g', 'de parmesan râpé ', '2c.à.c', 'c.à.c'...","['pinceau', 'saladier', 'Mixeur', 'bol']","[{'Étape 1': ""Faites dorer le pain, coupé en c..."
34731,"Crème anglaise réussie, onctueuse à souhait",25 min,50,Dessert,"['50g', 'de sucre ', '50cl', 'de lait demi-écr...","['casserole', 'Fouet cuisine', 'saladier', 'Cu...","[{'Étape 1': ""Porter à ébullition le lait en a..."


In [6]:
def split_quantity_units(text):
    if re.match(r"^\d.*[a-zA-Z]$",text) is not None:
        my_list = re.split(r'(\d+)', text)
        return " ".join(my_list)
    return text

In [7]:
def rm_duplicated_dgt(text:str):
    if text.isnumeric() and len(text)%2==0 and text[:int(len(text)/2)]==text[int(len(text)/2):]:
        return(text[:int(len(text)/2)])
    else:
        return(text)

# rm_duplicated_dgt(text)

In [8]:
def spacy_prep(texts):
    cleaned_texts = []
    docs = list(nlp.pipe(texts))
    for doc in docs:
        cleaned_texts.append(" ".join([token.text for token in doc if not token.is_punct
                                                                            and not token.is_stop
                                                                            ]))
    return cleaned_texts

In [9]:
def join_qty_ingredients(ingredients:str):
    list_ingredients = ast.literal_eval(ingredients)
    new_ingredients = []
    for i in range(len(list_ingredients)-1):
        if bool(re.search(r'\d', list_ingredients[i])):
            list_ingredients[i] = rm_duplicated_dgt(list_ingredients[i])
            list_ingredients[i] = split_quantity_units(list_ingredients[i])
            if list_ingredients[i+1] != list_ingredients[i].split(" ")[-1] and list_ingredients[i+1] != "":
                new_ingredients.append(" ".join([list_ingredients[i],list_ingredients[i+1]]))
            else:
                if i+2 <= len(list_ingredients):
                    new_ingredients.append(" ".join([list_ingredients[i],list_ingredients[i+2]]))
    return "\n".join(new_ingredients)

In [10]:
def format_recipe(nom,ingredients,ustensiles,Etapes):
    text = f"{str(nom)}\nIngrédients : \n{str(ingredients)}\nUstensiles :\n{str(ustensiles)}\nInstructions :\n {str(Etapes)}"
    return(text)

In [10]:
unit_abbreviation = {
    "c.à.s" : "cuillère à soupe",
    "c.à.c" : "cuillère à café",
    "g"     : "gramme",
    "l"     : "litre",
    "cl"    : "centilitre",
    "ml"    : "millilitre",
    "kg"    : "kilogramme",
}

In [11]:
print(join_qty_ingredients(data["Ingredients"][20]))

 1 kg de champignon « trompettes de la mort »
 2 c.à.s d'huile d'olive 
 1 gousse d'ail hachée
 1 ⁄ 2 botte de persil haché


In [12]:
def remove_accents(text:str):
    text = re.sub(r'[àáâãäå]', 'a', text)
    text = re.sub(r'[èéêë]', 'e', text)
    text = re.sub(r'[ìíîï]', 'i', text)
    text = re.sub(r'[òóôõö]', 'o', text)
    text = re.sub(r'[ùúûü]', 'u', text)
    return(text)

In [29]:
instructions = ast.literal_eval(data["Etapes"][0])
# print("\n".join(instructions))
instructions = [list(d.values())[0] for d in instructions]
print("\n".join(instructions))

Enlever l'écorce et les pépins du potimarron (il n'est pas obligatoire de le peler, mais dans ce cas, le choisir bio et bien le laver) puis couper la chair en gros morceaux.
Eplucher les pommes de terre puis les couper en gros morceaux.
Faire revenir les oignons et l'ail hachés dans un peu de beurre à feu doux.
Ajouter les pommes de terre et le potimarron, faire revenir 5 min.
Couvrir le tout d'eau, ajouter le bouillon de poule (émietté), le curry, la muscade, la feuille de laurier et laisser mijoter environ 30 min à feu doux.
Lorsque les légumes sont cuits (vérifier à l'aide d'un couteau), mixer votre préparation.
Vérifier l'assaisonnement, saler, poivrer selon votre goût puis ajouter la crème fraîche et laisser mijoter encore 2 ou 3 min. Bon appétit !


In [17]:
recipes = []
TEXT_FEATURES = ["Nom","Ingredients","Ustensiles","Etapes"]
CATEGORICAL_FEATURES = ["Type"]
for i in tqdm(range(2)):
    nom,ingredients,ustensiles,Etapes = (data[key][i] for key in TEXT_FEATURES)
    print(format_recipe(nom,ingredients,ustensiles,Etapes))
    # recipes.append(format_recipe(nom,ingredients,ustensiles,Etapes))



100%|██████████| 2/2 [00:00<00:00, 1997.29it/s]

soupe veloutée de potimarron et pommes de terre
Ingrédients : 
['22', 'pommes de terre ', '22', 'oignons hachés', '3gousses', "d'ail hachées", '10cl', 'de crème fraîche ', '1c.à.c', 'c.à.c', 'de curry ', '1pincée', 'de muscade ', '11', 'feuille de laurier ', '', 'sel ', '', 'poivre ', '11', 'potimarron ', '11', 'cube de bouillon de poule']
Ustensiles :
['louche', 'mixeur plongeant', 'Blender chauffant', 'Couteau', 'économe', 'pôele', 'Couvercle', 'Mixeur', 'Mijoteuse électrique', 'balance de cuisine']
Instructions :
 [{'Étape 1': "Enlever l'écorce et les pépins du potimarron (il n'est pas obligatoire de le peler, mais dans ce cas, le choisir bio et bien le laver) puis couper la chair en gros morceaux."}, {'Étape 2': 'Eplucher les pommes de terre puis les couper en gros morceaux.'}, {'Étape 3': "Faire revenir les oignons et l'ail hachés dans un peu de beurre à feu doux."}, {'Étape 4': 'Ajouter les pommes de terre et le potimarron, faire revenir 5 min.'}, {'Étape 5': "Couvrir le tout d'e

In [31]:
def clean_instructions(instructions:str):
    instruct = ast.literal_eval(instructions)
    instruct = [list(d.values())[0] for d in instruct]
    return instruct


In [32]:
def preprocessing_fn(data:pd.DataFrame):
    # data = df.copy()
    TEXT_FEATURES = ["Nom","Ingredients","Ustensiles","Etapes"]
    CATEGORICAL_FEATURES = ["Type"]
    
    for key in tqdm(TEXT_FEATURES):
        data[key] = data[key].apply(lambda x:x.lower())
        data[key] = data[key].apply(lambda x: remove_accents(x))
        if key == "Ingredients":
            data[key] = data[key].apply(lambda x:join_qty_ingredients(x))
        if key == "Etapes":
            data[key] = data[key].apply(lambda x:clean_instructions(x))

        data[key] = spacy_prep(data[key].to_numpy())

    recipes = []
    for i in tqdm(range(len(data))):
        nom,ingredients,ustensiles,Etapes = (data[key][i] for key in TEXT_FEATURES)
        recipes.append(format_recipe(nom,ingredients,ustensiles,Etapes))
    data["recettes"] = recipes

    for key in tqdm(CATEGORICAL_FEATURES):
        features =data[key].unique()
        encoder = LabelEncoder()
        encoder.fit(features)
        data[key] = encoder.transform(data["Type"])

   
    return data

In [33]:
cleaned_data = preprocessing_fn(data)

  0%|          | 0/4 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [59]:
cleaned_data

,Nom,Duree,Personnes,Type,Ingredients,Ustensiles,Etapes,recettes
0,soupe veloute potimarron pomme terre,45 min,4,4,2 pomme terre 2 oignon hache 3 gousse ail ha...,louche mixeur plonger blender chauffer couteau...,etape 1 enlever ecorce pepin potimarron obliga...,soupe veloute potimarron pomme terre 2 pomme ...
1,soupe oignon,45 min,4,4,4 oignon 50 gramme beurre 1 c.a. huile 1...,louche mixeur plonger blender chauffer saladie...,etape 1 pelez emincer oignon etape 2 faire rev...,soupe oignon 4 oignon 50 gramme beurre 1 ...
2,veloute potiron,1 h 15 min,4,4,1 oignon 1 quartier potiron 1 brique creme f...,louche mixeur plonger couteau cocotte fonte po...,etape 1 couper chair potiron gros de etape 2 c...,veloute potiron 1 oignon 1 quartier potiron...
3,veloute butternut cannelle,35 min,4,4,1 oignon 1 c.a. creme frais 1 pincee canne...,louche mixeur plonger couteau cuillere bois po...,etape 1 couper butternut gros cube etape 2 cou...,veloute butternut cannelle 1 oignon 1 c.a. ...
4,tarte poireau lardon,50 min,6,4,250 gramme farine 140 gramme beurre 2 c....,moule tarte four couteau pinceau rape poele sa...,etape 1 prechauffer four 210 degre c thermosta...,tarte poireau lardon 250 gramme farine 14...
...,...,...,...,...,...,...,...,...
47335,liqueur genepi,10 min,1,2,40 morceau sucre 40 brin genepi 1 l alco...,entonnoir balance cuisine,etape 1 transvaser litre alcool gros bouteille...,liqueur genepi 40 morceau sucre 40 brin g...
47336,caipirinha,3 min,1,2,1 dose jus citron vert frais 15 millilitre j...,cuillere bois shaker,etape 1 shaker melanger ingredient glace pilee...,caipirinha 1 dose jus citron vert frais 15 ...
47337,kombucha maison citron gingembre,-,1,2,1 morceau gingembre raper 100 gramme sucre...,casserole balance cuisine,etape 1 commencer realiser sirop chauffer eau ...,kombucha maison citron gingembre 1 morceau ...
47338,gin tonic,5 min,1,2,4 cl gin 2 rondelle citron 8 cl tonic,,etape 1 verser glaçon verre etape 2 ajouter gi...,gin tonic 4 cl gin 2 rondelle citron 8 ...


In [60]:
cleaned_data.to_csv(os.path.join(repo_dir,"data","cleaned_data.csv"),encoding="utf-8",sep=",", index=False)

In [40]:
txt_filename = os.path.join(repo_dir,"data","recettes.txt")
# start_token = '<|startofrecipe|>'
# end_token = '<|endofrecipe|>'

with open(txt_filename, 'w',encoding='utf-8') as fp:
    for item in data["recettes"].to_list():
        fp.write("%s\n\n"%item.strip())
    print('Done')

Done
